In [7]:
import gradio as gr
import ast
from pynput import keyboard
import time
import json
from pynput.keyboard import Key, Controller

In [2]:
def update_macros(action, keys, macro_to_delete, current_macros):
    # Check if current_macros is a string representation of a dictionary, and convert it if so
    if isinstance(current_macros, str) and current_macros.strip():
        # Safely convert the string representation back to a dictionary
        current_macros = ast.literal_eval(current_macros)
    else:
        # Initialize as an empty dictionary if not a valid string representation
        current_macros = {}
    
    # Check if a user entered a macro to delete
    if macro_to_delete and macro_to_delete in current_macros:
        del current_macros[macro_to_delete]  
    elif action and keys:  # Ensure non-empty input
        # Update or add the macro definition
        newlist = key_name.copy()
        newkey = '  +  '.join(newlist)
        current_macros[action] = {'keys': newkey}
    # Format the macros for display in a readable format
    macros_text = ""
    for act, vals in current_macros.items():
        macros_text += f"{act}  =>  {vals['keys']}\n"

    clear_list()

    # Return both the formatted string for display and the updated macros dictionary, Clear the display for action_input, keys_input, and macro_to_delte
    return macros_text, str(current_macros), "", "", "Keys to Simulate"

In [8]:
with open('macros.json', 'r') as file:
    data = json.load(file)

macros = data["macros"]

keyboard = Controller()

macro_functions = []
for macro in macros:
    keycodes = macro["keycodes"]
    delay = macro["delay"]

    macro_function = lambda keycodes=keycodes, delay=delay: [
        (keyboard.press(Key.enter) if keycode == Key.enter else 
         keyboard.press(Key.space) if keycode == Key.space else 
         keyboard.press(chr(keycode))) for keycode in keycodes
    ] + [
        (keyboard.release(Key.enter) if keycode == Key.enter else 
         keyboard.release(Key.space) if keycode == Key.space else 
         keyboard.release(chr(keycode))) for keycode in keycodes
    ] + [
        time.sleep(delay)
    ]

    macro_functions.append(macro_function)


FileNotFoundError: [Errno 2] No such file or directory: 'macros.json'

In [3]:
key_list = []
key_name = []

key_delays = []  # Store delays directly
last_release_time = None  # Track the last key release time

def read_keys():
    global last_release_time
    def on_press(key):
        global last_release_time
        
        try:
            print('alphanumeric key {0} pressed'.format(key.char))
            if key == keyboard.Key.esc:
                last_release_time = None
                return False
            current_time = time.time()
            if last_release_time is not None:
                delay = current_time - last_release_time
                key_delays.append(delay)
        except AttributeError:
            current_time = time.time()
            if last_release_time is not None:
                delay = current_time - last_release_time
                key_delays.append(delay)
            print('special key {0} pressed'.format(key))
        
    def on_release(key):
        global last_release_time
        try:
            last_release_time = time.time()
            key_list.append(key.char)
            key_name.append(key.char)
        except AttributeError:
            if key == keyboard.Key.esc:
                last_release_time = None
                return False
            last_release_time = time.time()
            key_str = getattr(key, 'name', str(key))
            key_list.append(key.value)
            key_name.append(key_str)
        print(key_list)
        print(key_delays)
    with keyboard.Listener(
        on_press=on_press,
        on_release=on_release) as listener:
        listener.join()

    return "Ended Recording"

def clear_list():
    key_list.clear()
    key_name.clear()
    key_delays.clear()


In [4]:
def update_key_btn():
    return "Now Recording Press The 'esc' Key To Stop"

In [5]:
with gr.Blocks() as settings_page:
    gr.Markdown("# Macro Setting")
    gr.Markdown("""
    Configure your audio command macros here. Enter a speech command, enter your key sequence, and set the timing.
    """)

    with gr.Row():
        # Create a textbox for the speech command
        action_input = gr.Textbox(label="Speech Command") 
        # Create a textbox for the key
        keys_input = gr.Button("Keys to Simulate")

    # Gets the macro the user wants to delete
    macro_to_delete = gr.Textbox(label="Type the Speech Command to Delete")
    # Creates a button to update or add the new settings
    update_btn = gr.Button("Update/Add Macro") 
    
    # Display for the current macros
    current_macros_display = gr.Textbox(label="Current Macros", value="", lines=10, interactive=False) 
    macros_state = gr.State('{}')
    # Runs when the button is clicked updating the current macros display
    keys_input.click(update_key_btn, outputs=keys_input)
    keys_input.click(read_keys, outputs=keys_input)
    update_btn.click(update_macros, 
                     inputs=[action_input, keys_input,macro_to_delete, macros_state], 
                     outputs=[current_macros_display, macros_state, action_input, macro_to_delete, keys_input])


In [6]:
settings_page.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


alphanumeric key r pressed
['r']
[]
alphanumeric key e pressed
['r', 'e']
[1.7064530849456787]
alphanumeric key e pressed
['r', 'e', 'e']
[1.7064530849456787, 2.8210887908935547]
special key Key.esc pressed
